<a href="https://colab.research.google.com/github/AbrarAhmed647/CarZam--A-Vehicular-Classification-using-EDNA-ML/blob/main/Fall_'22_CarZam_Part_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading

In [1]:
import os
if not os.path.exists('./original_tool_image.zip'):
  ! wget -O original_tool_image.zip https://www.dropbox.com/s/pha9yzdfkmzoqob/original_tool_images.zip?dl=0

--2022-11-22 19:16:43--  https://www.dropbox.com/s/pha9yzdfkmzoqob/original_tool_images.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/pha9yzdfkmzoqob/original_tool_images.zip [following]
--2022-11-22 19:16:44--  https://www.dropbox.com/s/raw/pha9yzdfkmzoqob/original_tool_images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf019918550871e622ba5c525b1.dl.dropboxusercontent.com/cd/0/inline/BxRg6aJ131SGAgXnvPEoaEK9LAyBRmCP48mWhryfWdtM5xxa2kOVN5cBOcOUPcu1Alusn4COFuMrhNpGpXCPsIvyxLilU2QADF8xdPsscdTtrvpO2GMsHBFia_d3Emo-rrvDTqtM2yrN6m78p5xtXhionx2p8KpA9IVNlocSihaveg/file# [following]
--2022-11-22 19:16:44--  https://ucf019918550871e622ba5c525b1.dl.dropboxusercontent.com/cd/0/inline/BxRg6aJ131SGAgXnvPEoaEK9LAyBRmCP48mW

# Setup

## Git Clone

### From Source

In [2]:
! rm -rf -- GLAMOR

In [3]:
! git clone -b master https://github.com/asuprem/GLAMOR

Cloning into 'GLAMOR'...
remote: Enumerating objects: 8027, done.
remote: Counting objects: 100% (2514/2514), done.
remote: Compressing objects: 100% (797/797), done.
remote: Total 8027 (delta 1569), reused 2350 (delta 1446), pack-reused 5513
Receiving objects: 100% (8027/8027), 2.18 MiB | 2.52 MiB/s, done.
Resolving deltas: 100% (5270/5270), done.


In [4]:
!pip install -e GLAMOR/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/GLAMOR
     |████████████████████████████████| 1.3 MB 5.9 MB/s 
  Running setup.py develop for ednaml


###  From PyPi

In [5]:
#! python -V

In [6]:
#! pip3 install --pre ednaml==0.1.4

# Restart Notebook to Finish EdnaML Installation

# Setting up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import ednaml
import glob, os
#from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

'1.12.1+cu113'

## Definitions: Crawler

In [3]:
# Here we define our custom model class
from ednaml.crawlers import Crawler
from zipfile import ZipFile # might be useful in unzipping!

class CarZamCrawler(Crawler):
  def __init__(self, logger, file_name = "original_tool_images.zip", **kwargs): # Add your own arguments if needed!
    self.classes = {}
    self.metadata = {}
    self.metadata["train"] = {}
    self.metadata["test"] = {}
    self.metadata["val"] = {}
    self.metadata["train"]["crawl"] = []  # <------ THIS NEEDS TO BE POPULATED
    self.metadata["test"]["crawl"] = []   # <------ THIS NEEDS TO BE POPULATED
    self.metadata["val"]["crawl"] = []    # <------ THIS NEEDS TO BE POPULATED

    # YOUR CODE HERE ------ POPULATE self.classes and self.metadata's empty lists ---
    from zipfile import ZipFile
    file_name = "original_tool_image.zip"
    fdest= "unzipped"
    if not os.path.exists(fdest):
      with ZipFile(file_name, 'r') as zip: 
          # extract all files to another directory
          zip.extractall(fdest)
    fllist = glob.glob(os.path.join(fdest, "original_tool_images/*.jpg"))
    #tuple_prelim = [self.getinittuple(item) for item in fllist]
    temp=[]
    #ans=[]
    tokeep = ["Convertible", "Coupe", "Crossover" "Diesel", "Hybrid", "Sedan", "SUV", "Wagon"]
    tuple_expanded=[]
    for item in fllist:
      tuple_prelim=os.path.splitext(os.path.basename(item))[0].split(" "), item
      #print(tuple_prelim)
      if(len(tuple_prelim[0])==5):
        for i in tuple_prelim[0]:
          temp.append(i)
        temp.append(tuple_prelim[1])
        my_tuple=tuple(temp)
        tuple_expanded.append(my_tuple)
        temp=[]
      elif(len(tuple_prelim[0])==6):
        if(tuple_prelim[0][0]) in tokeep:
          for i in tuple_prelim[0][:4]:
            temp.append(i)
          temp.append(tuple_prelim[0][4]+tuple_prelim[0][5])
          temp.append(tuple_prelim[1])
          my_tuple=tuple(temp)
          tuple_expanded.append(my_tuple)
          temp=[]
        else:
          temp.append(tuple_prelim[0][0]+tuple_prelim[0][1])
          for i in tuple_prelim[0][2:]:
            temp.append(i)
          temp.append(tuple_prelim[1])
          my_tuple=tuple(temp)
          tuple_expanded.append(my_tuple)
          temp=[]
  #print(ans)
      elif(len(tuple_prelim[0])==7):
        temp.append(tuple_prelim[0][0]+tuple_prelim[0][1])
        for i in tuple_prelim[0][2:5]:
          temp.append(i)
        temp.append(tuple_prelim[0][5]+tuple_prelim[0][6])
        temp.append(tuple_prelim[1])
        my_tuple=tuple(temp)
        tuple_expanded.append(my_tuple)
        temp=[]
    
    import random
    random.seed(3456)
    random.shuffle(tuple_expanded)

    splits = 0.8
    train_sets = int(len(tuple_expanded)*0.8)
    val_sets = int(len(tuple_expanded)*0.1)

    

    # structure:  (path, type, color, year, make)
    # idx           0     1     2     3     4
    types = list(set([item[1] for item in tuple_expanded]))
    colors = list(set([item[2] for item in tuple_expanded]))
    years = list(set([item[3] for item in tuple_expanded]))
    makes = list(set([item[4] for item in tuple_expanded]))

    self.classes["vtype"] = len(types)
    self.classes["color"] = len(colors)
    self.classes["year"] = len(years)
    self.classes["make"] = len(makes)

    self.type_lookup = {item:idx for idx,item in enumerate(types)}
    self.color_lookup = {item:idx for idx,item in enumerate(colors)}
    self.year_lookup = {item:idx for idx,item in enumerate(years)}
    self.make_lookup = {item:idx for idx,item in enumerate(makes)}

    tuple_expanded = [(item[0], self.type_lookup[item[1]], self.color_lookup[item[2]], self.year_lookup[item[3]], self.make_lookup[item[4]]) for item in tuple_expanded]

    self.metadata["train"]["crawl"] = tuple_expanded[:train_sets]
    self.metadata["val"]["crawl"] = tuple_expanded[train_sets:val_sets]
    self.metadata["test"]["crawl"] = tuple_expanded[train_sets+val_sets:]


    # -------------------------------------------------------------------------------

    self.metadata["train"]["classes"] = self.classes
    self.metadata["test"]["classes"] = self.classes
    self.metadata["val"]["classes"] = self.classes

  def getinittuple(self, item):
    return (os.path.splitext(os.path.basename(item))[0].split(" "), item)

    


## Testing the Crawler

In [4]:
kwargs = {
    "logger" : None,
    "file_name" : "original_tool_images.zip",
    # add any other kwargs here...
}

In [5]:
crawler = CarZamCrawler(**kwargs)

In [6]:
crawler.classes # You should get the classes here

{'vtype': 22, 'color': 27, 'year': 56, 'make': 315}

In [7]:
crawler.metadata["train"]["crawl"]  # You should get the list of tuples here

[('Coupe', 13, 1, 9, 136),
 ('SportsCar', 19, 7, 11, 172),
 ('Sedan', 10, 1, 22, 192),
 ('Coupe', 19, 4, 9, 136),
 ('Sedan', 19, 18, 37, 142),
 ('SUV', 3, 8, 7, 210),
 ('Wagon', 17, 7, 11, 81),
 ('SUV', 19, 18, 22, 97),
 ('Crossover', 17, 4, 11, 287),
 ('Wagon', 21, 25, 54, 284),
 ('LuxuryVehicle', 17, 14, 17, 266),
 ('SportsCar', 17, 4, 50, 293),
 ('Hybrid', 13, 0, 31, 275),
 ('LuxuryVehicle', 13, 8, 17, 154),
 ('Van', 10, 1, 9, 277),
 ('LuxuryVehicle', 17, 22, 1, 27),
 ('Wagon', 21, 25, 50, 98),
 ('Coupe', 2, 14, 50, 293),
 ('SportsCar', 17, 8, 34, 184),
 ('Crossover', 3, 8, 41, 108),
 ('Coupe', 17, 25, 38, 272),
 ('SUV', 3, 1, 26, 157),
 ('SportsCar', 10, 4, 11, 172),
 ('PickupTruck', 18, 4, 7, 128),
 ('SportsCar', 10, 0, 31, 10),
 ('SUV', 10, 7, 17, 107),
 ('SUV', 10, 25, 31, 106),
 ('SUV', 3, 4, 37, 43),
 ('SUV', 13, 18, 52, 14),
 ('Hybrid', 21, 7, 7, 19),
 ('Wagon', 17, 18, 26, 93),
 ('Coupe', 17, 25, 50, 125),
 ('Wagon', 3, 18, 52, 41),
 ('SportsCar', 17, 8, 30, 4),
 ('Crossover

# Statistics

In [8]:
import matplotlib.pyplot as plt

In [ ]:
# Write code to collect info on # makes, models, year, type